In [1]:
import json
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import pandas as pd
from datasets import Dataset
import torch

2025-05-24 05:17:59.911001: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748063880.089004      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748063880.144501      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "train_tamil1.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "uom200501p/tamilqa-csv",
  file_path,
)

/tmp/ipykernel_35/3593000134.py:8: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


In [4]:
df

,id,title,context,question,answers_text,answers_answer_start
0,5a87d4d219b91f001a626e48,வெற்றிடம்,வெற்றிட வெளிப்பாட்டைக் காட்டிலும் விரைவான டிகம...,ஆக்ஸிஜன் நுகர்வு அதிகரிப்பது நுரையீரலுக்கு என்...,[],[]
1,5a87d4d219b91f001a626e49,வெற்றிடம்,வெற்றிட வெளிப்பாட்டைக் காட்டிலும் விரைவான டிகம...,மூச்சுக்குழாய் வழியாக காற்றோட்டம் செவிப்பறைகளை...,[],[]
2,5a87d4d219b91f001a626e4a,வெற்றிடம்,வெற்றிட வெளிப்பாட்டைக் காட்டிலும் விரைவான டிகம...,உங்கள் மூச்சைப் பிடித்துக் கொண்டால் மூச்சுக் க...,[],[]
3,5a87d4d219b91f001a626e4b,வெற்றிடம்,வெற்றிட வெளிப்பாட்டைக் காட்டிலும் விரைவான டிகம...,மூச்சுக்குழாய் வழியாக காற்றோட்டம் என்று என்ன அ...,[],[]
4,572eaab5cb0c0d14000f1428,வெற்றிடம்,பிளாட்டோவுக்குப் பிறகு கிட்டத்தட்ட இரண்டாயிரம்...,இயற்கையில் உள்ள வெற்றிடங்களைப் பற்றி டெஸ்கார்ட...,"['இயற்கையில் ஒரு வெற்றிடம் ஏற்படாது,']",[269]
...,...,...,...,...,...,...
119042,5a6bf3af4eec6b001a80a613,வேட்டையாடுதல்,ஒரு வேட்டையாடும் அதன் இரை இனங்கள் மீதான தாக்கத...,வனவிலங்கு பாதுகாவலர்கள் இரையை மட்டுப்படுத்தினா...,[],[]
119043,5a6bf3af4eec6b001a80a614,வேட்டையாடுதல்,ஒரு வேட்டையாடும் அதன் இரை இனங்கள் மீதான தாக்கத...,ஒரு புதிய இரை இனத்தை அறிமுகப்படுத்துவது பலவீனம...,[],[]
119044,5726f545708984140094d6fd,வேட்டையாடுதல்,அறிமுகப்படுத்தப்பட்ட உயிரினங்களைக் கட்டுப்படுத...,இயற்கை வேட்டையாடுபவர்கள் பயிர்களுக்கு எவ்வாறு ...,['பயிர்களுக்கு ஏற்படும் சேதத்தை குறைக்கும் வழி...,[666]
119045,572fdefb947a6a140053cd8d,ஆண்டெனா_(ரேடியோ),மிகவும் பரவலாகப் பயன்படுத்தப்படும் ஆண்டெனா வகை...,ஆண்டெனா அச்சுக்கு செங்குத்தாக இரண்டு லோப்களை வ...,['கிடைமட்ட இருமுனை'],[244]


In [5]:
dataset = Dataset.from_pandas(df)

In [44]:
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("RajeevanL/distilled_XLMRoberta_153_v5")
model = AutoModelForQuestionAnswering.from_pretrained("RajeevanL/distilled_XLMRoberta_153_v5")

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/614M [00:00<?, ?B/s]

In [12]:
context = """
வெற்றிட வெளிப்பாட்டைக் காட்டிலும் விரைவான டிகம்ப்ரஷன் மிகவும் ஆபத்தானது. பாதிக்கப்பட்டவர் தனது மூச்சைப் பிடித்துக் கொள்ளாவிட்டாலும், நுரையீரலின் நுண்ணிய அல்வியோலியின் அபாயகரமான சிதைவைத் தடுக்க மூச்சுக் குழாய் வழியாக வெளியேறுவது மிகவும் மெதுவாக இருக்கும். காதுகுழாய்கள் மற்றும் சைனஸ்கள் விரைவான டிகம்பரஷ்ஷனால் சிதைக்கப்படலாம், மென்மையான திசுக்கள் சிராய்ப்பு மற்றும் இரத்தத்தை கசியும், மேலும் அதிர்ச்சியின் அழுத்தம் ஆக்ஸிஜன் நுகர்வு ஹைபோக்ஸியாவுக்கு வழிவகுக்கும். விரைவான டிகம்பரஷ்ஷனால் ஏற்படும் காயங்கள் பரோட்ராமா என்று அழைக்கப்படுகின்றன. ௧௩ கேபிஏ (௧௦௦ டோர்) அழுத்தம் குறைவது, அது படிப்படியாக இருந்தால் எந்த அறிகுறிகளையும் உருவாக்காது, அது திடீரென்று ஏற்பட்டால் அது ஆபத்தானது.

மேலும், மரபணுக்களில் ஏற்பட்ட மாற்றங்கள் உடல் செயல்பாடுகளில் பெரும் பாதிப்பை ஏற்படுத்தலாம். குறைந்த அழுத்தநிலை நீண்ட நேரம் தொடர்ந்தால் திசுக்கள் நசுக்கும் வாய்ப்பு அதிகரிக்கிறது. இவ்வாறு ஏற்பட்ட ஆழமான பிசின் அழுத்தம் நரம்பியல் பாதிப்புகள் மற்றும் இரத்த ஓட்டத்தின் குறைபாடுகளை உருவாக்கும். விரைவில் மருத்துவமனையில் சென்று பரிசோதனை செய்வது அவசியம்.

இதுபோன்ற சூழ்நிலைகளில் பயணிகள் மற்றும் விரும்பத்தக்க அவசர சேவைகள் பாகுபாடு செய்யப்பட வேண்டும். மருத்துவ ஆலோசனைகள் அவசியமானதாக இருக்கின்றன மற்றும் அவை பாதுகாப்பு நடைமுறைகளுடன் பின்பற்றப்பட வேண்டும். உடல்நலம் பாதுகாப்பதற்கான முறைகள் சிறப்பாக திட்டமிடப்பட்டிருக்க வேண்டும்.
"""


In [13]:
encoding = tokenizer(context, return_tensors="pt")

In [14]:
tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"][0])

In [15]:
len(tokens)

341

In [29]:
import ast

def preprocess_function(examples):
    questions = [q.strip() if q is not None else "" for q in examples["question"]]
    contexts = [c if isinstance(c, str) else "" for c in examples["context"]]
    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        # Get raw answers (may be list or stringified list)
        answer_starts = examples["answers_answer_start"][i]
        answer_texts = examples["answers_text"][i]

        # If strings like '[123]' or "['answer']", parse to actual list
        if isinstance(answer_starts, str):
            try:
                answer_starts = ast.literal_eval(answer_starts)
            except Exception:
                answer_starts = []
        if isinstance(answer_texts, str):
            try:
                answer_texts = ast.literal_eval(answer_texts)
            except Exception:
                answer_texts = []

        # If empty after parsing, treat as no answer
        if not answer_texts or not answer_starts:
            start_positions.append(0)
            end_positions.append(0)
            continue

        # Convert to int safely
        try:
            start_char = int(answer_starts[0])
        except Exception:
            start_positions.append(0)
            end_positions.append(0)
            continue

        end_char = start_char + len(answer_texts[0])
        sequence_ids = inputs.sequence_ids(i)

        # Find context token start/end indices
        context_start = None
        context_end = None
        for idx, seq_id in enumerate(sequence_ids):
            if seq_id == 1 and context_start is None:
                context_start = idx
            if seq_id == 1:
                context_end = idx

        if context_start is None or context_end is None:
            start_positions.append(0)
            end_positions.append(0)
            continue

        # Check if answer inside the context span
        if offsets[context_start][0] > end_char or offsets[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
            continue

        # Find start token index
        idx = context_start
        while idx <= context_end and offsets[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        # Find end token index
        idx = context_end
        while idx >= context_start and offsets[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [30]:
# Tokenize
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/119047 [00:00<?, ? examples/s]

In [32]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 119047
})

In [33]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 95237
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 23810
    })
})

In [39]:
# Training args
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[], 
)

In [40]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    data_collator=default_data_collator,
    tokenizer=tokenizer,
)

# Train model
trainer.train()

/tmp/ipykernel_35/1566634480.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.503600,3.396453
2,3.320100,3.346415


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=5954, training_loss=3.4356312827051667, metrics={'train_runtime': 3511.6259, 'train_samples_per_second': 54.241, 'train_steps_per_second': 1.696, 'total_flos': 1.1068342253171712e+16, 'train_loss': 3.4356312827051667, 'epoch': 2.0})

In [41]:
# Save model and tokenizer locally in Kaggle working directory
model.save_pretrained("/kaggle/working/fine_tuned_tamil_qa_model")
tokenizer.save_pretrained("/kaggle/working/fine_tuned_tamil_qa_model")

('/kaggle/working/fine_tuned_tamil_qa_model/tokenizer_config.json',
 '/kaggle/working/fine_tuned_tamil_qa_model/special_tokens_map.json',
 '/kaggle/working/fine_tuned_tamil_qa_model/sentencepiece.bpe.model',
 '/kaggle/working/fine_tuned_tamil_qa_model/added_tokens.json',
 '/kaggle/working/fine_tuned_tamil_qa_model/tokenizer.json')

In [ ]:
# Push to Hugging Face Hub
from huggingface_hub import login
import os

# Manually set your Hugging Face token here as a string
hf_token = "YOUR_HUGGING_FACE_TOKEN"
login(token=hf_token)

In [43]:
model.push_to_hub("RajeevanL/distilled_XLMRoberta_153_v6")
tokenizer.push_to_hub("RajeevanL/distilled_XLMRoberta_153_v6")


model.safetensors:   0%|          | 0.00/614M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RajeevanL/distilled_XLMRoberta_153_v2/commit/499ee4e1fd4ee66dc854f7251fb6126e3eabde88', commit_message='Upload tokenizer', commit_description='', oid='499ee4e1fd4ee66dc854f7251fb6126e3eabde88', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RajeevanL/distilled_XLMRoberta_153_v2', endpoint='https://huggingface.co', repo_type='model', repo_id='RajeevanL/distilled_XLMRoberta_153_v2'), pr_revision=None, pr_num=None)